In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,
                                        CSVLogger, EarlyStopping)
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Flatten, Dense)
from tensorflow.keras.models import Model
import datetime
import pandas as pd
import pickle
import demo_model
import numpy as np

model_name = "demo_model"

# load demo data

In [7]:
demo_agesex_features = pd.read_csv("demo_data/demo_asfeats.csv") # ecgId (used for path) + tabular features 
demo_ecgs = ["ecg_%d"%i for i in range(len(demo_agesex_features))] # paths to ecg files, assuming they're numpy arrays
demo_labels = pd.read_csv("demo_data/demo_labels.csv",index_col=0) 

# Create generators and train

In [8]:

training_generator = demo_model.DataGenerator(demo_ecgs, demo_labels,n_classes=demo_labels.shape[1],age_sex=demo_agesex_features)
validation_generator = demo_model.DataGenerator(demo_ecgs, demo_labels,n_classes=demo_labels.shape[1],age_sex=demo_agesex_features)

In [9]:
loss = 'binary_crossentropy'
lr = 0.005
batch_size = 64
opt = Adam(lr)
callbacks = [ReduceLROnPlateau(monitor='val_loss',
                               factor=0.05,
                               patience=2,
                               min_lr=lr / 100),
             EarlyStopping(patience=5,min_delta=0.00001)]
METRICS = [
      keras.metrics.AUC(multi_label=True,),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model = demo_model.get_model_agsx(label_df.shape[1])
model.compile(loss=loss, optimizer=opt, metrics=METRICS)

# Save the BEST model
callbacks += [ModelCheckpoint('./models/demo/%s'%model_name, save_best_only=True)]
model.fit(x= training_generator,
                    validation_data=validation_generator,
                     epochs=70,
                    callbacks=callbacks,
                    workers=1)

2023-04-04 14:14:34.375865: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-04 14:14:34.376660: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-04 14:14:34.435434: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-04 14:14:34.435731: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.90GiB deviceMemoryBandwidth: 451.17GiB/s
2023-04-04 14:14:34.435755: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-04-04 14:14:34.437472: I tensorflow/stream_executor/platfor

NameError: name 'label_df' is not defined